# Groq API 써보기

Groq의 빠른 추론 엔진을 LangChain에서 체험하는 코드를 소개합니다.    
langchain_groq를 통해 쉽게 연결할 수 있습니다.

In [1]:
!pip install langchain langchain_groq langchain_community pymupdf pypdf beautifulsoup4 requests

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 11.4 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 33.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 56.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.7/300.7 KB 36.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 186.0/186.0 KB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.2/413.2 KB 35.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 332.8/332.8 KB 34.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 57.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 110.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 KB 27.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 205.1/205.1 KB 44.8 MB/s eta 0:00:

현재 사용 가능한 주요 모델과 API 제한은 아래와 같습니다.


| ID                                     | Requests per Minute | Requests per Day | Tokens per Minute | Tokens per Day |
|----------------------------------------|---------------------|------------------|-------------------|----------------|
| gemma-7b-it                            | 30                  | 14,400           | 15,000            | 500,000        |
| gemma2-9b-it                           | 30                  | 14,400           | 15,000            | 500,000        |
| llama-3.1-70b-versatile                | 30                  | 14,400           | 18,000            | 500,000        |
| llama-3.1-8b-instant                   | 30                  | 14,400           | 20,000            | 500,000        |
| llama-3.2-11b-text-preview             | 30                  | 7,000            | 7,000             | 500,000        |
| llama-3.2-11b-vision-preview           | 30                  | 7,000            | 7,000             | 500,000        |
| llama-3.2-1b-preview                   | 30                  | 7,000            | 7,000             | 500,000        |
| llama-3.2-3b-preview                   | 30                  | 7,000            | 7,000             | 500,000        |
| llama-3.2-90b-text-preview             | 30                  | 7,000            | 7,000             | 500,000        |
| llama-3.2-90b-vision-preview           | 15                  | 3,500            | 7,000             | 250,000        |
| llava-v1.5-7b-4096-preview             | 30                  | 14,400           | 30,000            | (No limit)     |
| mixtral-8x7b-32768                     | 30                  | 14,400           | 5,000             | 500,000        |


In [2]:
from langchain_groq import ChatGroq # Groq-LangChain 연결

from langchain_community.document_loaders import WebBaseLoader

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

import time

USER_AGENT environment variable not set, consider setting it to identify your requests.


간단한 체인을 만들고 실행합니다.

https://console.groq.com/keys 에서 키를 생성해야 합니다.

In [3]:
import os
os.environ['GROQ_API_KEY'] = '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' #기존 groq Key로 여러명이 동시에 동작시키면 차단될 수 있습니다. https://groq.com/ DEV CONSOLE에서 접속하여 여러분의 개별 groq Key를 넣어주세요.


chat = ChatGroq(
    temperature=0.1,
    model="llama-3.2-11b-vision-preview",
)

url = "https://wow.groq.com/why-groq/"
loader = WebBaseLoader(url)

docs = loader.load()
question = "Groq 엔진의 LLM 추론 속도가 빠른 이유는 무엇인가요? 한국어로 답변하세요."


system = "Answer the question from given contexts. Answer in Korean."
human = """
Context: {context}

---

Question: {question}
"""

prompt = ChatPromptTemplate.from_messages([("system", system), ("human", human)])

chain = prompt | chat | StrOutputParser()


start = time.time()
result = chain.invoke(
          {"context":docs[0].page_content,
            'question':question})
print(result)

end = time.time()
elapsed_time = end - start
print(f"Elapsed Time: {elapsed_time} seconds")

Groq 엔진의 LLM(Large Language Model) 추론 속도가 빠른 이유는 Groq Language Processing Unit(GPU)가 설계된 목적이 AI 추론과 언어 처리에 최적화된 것 때문입니다. GPU는 원래 그래픽 처리를 위해 설계되었지만, Groq LPU는 AI 추론과 언어 처리를 위해 설계되었습니다.

Groq LPU는 다음과 같은 특징을 가지고 있습니다.

1. **AI 추론에 최적화**: Groq LPU는 AI 추론을 위해 설계되었습니다. 따라서 AI 모델을 실행하는 데 최적화되어 있습니다.
2. **언어 처리에 최적화**: Groq LPU는 언어 처리를 위해 설계되었습니다. 따라서 언어 모델을 실행하는 데 최적화되어 있습니다.
3. **에너지 효율성**: Groq LPU는 에너지 효율성을 높여 AI 추론을 더 빠르게 수행할 수 있도록 설계되었습니다.
4. **가격 competitiveness**: Groq LPU는 가격이 저렴하여 AI 추론을 더 많은 사람들에게 접근할 수 있도록 설계되었습니다.

이러한 특징들로 인해 Groq 엔진의 LLM 추론 속도가 빠르게 수행될 수 있습니다.
Elapsed Time: 1.1311769485473633 seconds


## Summarization

요약은 LLM의 아주 중요한 기능 중 하나입니다.   
일반적으로, LLM의 Abstractive Summarization은 3개의 방법을 사용합니다.

- Stuff : 전체 코퍼스를 하나의 프롬프트에 넣고 요약 생성하기
- Map-Reduce : 코퍼스를 청크로 분리하고, 각 청크의 요약을 생성한 뒤 합치기
- Refine: 코퍼스를 청크로 분리하고, 순차적으로 읽으며 요약 업데이트하기



## Stuff

gemma 2 모델을 이용해 요약을 수행해 보겠습니다.

In [4]:
# Stuff
example_URL='https://arxiv.org/abs/1706.03762'

loader = WebBaseLoader(example_URL)
docs = loader.load()


gemma2 = ChatGroq(
    temperature=0,
    model="gemma2-9b-it",
)



summarize_prompt = ChatPromptTemplate.from_messages([
    ('system', '''Summarize the following paper in Korean.
Emphasize the uniqueness and contribution of the paper.
Answer should be in 10 sentences.
Please Answer in Korean.
'''),
    ('user', '''{text}''')])

print(len(docs[0].page_content))
summarize_prompt.format_messages(text=docs[0].page_content)

5950


[SystemMessage(content='Summarize the following paper in Korean.\nEmphasize the uniqueness and contribution of the paper.\nAnswer should be in 10 sentences.\nPlease Answer in Korean.\n', additional_kwargs={}, response_metadata={}),
 HumanMessage(content="\n\n [1706.03762] Attention Is All You Need\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n  \n\n\n\n\n\n\n\n\nSkip to main content\n\n\n\n\n\nWe gratefully acknowledge support from the Simons Foundation, member institutions, and all contributors.\nDonate\n\n\n\n\n\n > cs > arXiv:1706.03762\n  \n\n\n\n\n\nHelp | Advanced Search\n\n\n\n\nAll fields\nTitle\nAuthor\nAbstract\nComments\nJournal reference\nACM classification\nMSC classification\nReport number\narXiv identifier\nDOI\nORCID\narXiv author ID\nHelp pages\nFull text\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nopen search\n\n\n\n\n\n\nGO\n\n\n\nopen navigation menu\n\n\nquick links\n\nLogin\nHelp Pages\nAbout\n\n\n\n\n\n\n\n\n\n\n\n\nComputer Science > Computation 

In [ ]:
summarize_chain = summarize_prompt | gemma2 | StrOutputParser() #chain 만들기
summary = summarize_chain.invoke(docs[0].page_content)
print(summary)

이 논문 "Attention Is All You Need"는 순환 신경망이나 합성곱 신경망과 같은 복잡한 구조를 사용하지 않고, **단순히 Attention 메커니즘만을 기반으로** 기존의 기계 번역 모델을 뛰어넘는 성능을 보여주는 새로운 네트워크 아키텍처인 Transformer를 제안합니다. 

이 논문의 가장 큰 혁신은 **RNN이나 CNN을 사용하지 않고도 효과적인 시퀀스 전환 모델을 구축할 수 있다는 것을 증명**한 것입니다. 

Transformer는 Attention 메커니즘을 통해 입력 시퀀스의 모든 요소 간의 관계를 효과적으로 학습하여, 기존 모델보다 더 높은 번역 품질을 달성합니다. 

실험 결과, Transformer는 WMT 2014 영어-독일어 및 영어-프랑스어 번역 작업에서 기존 최고 성능을 뛰어넘는 결과를 보였습니다. 

특히, Transformer는 **병렬 처리가 가능**하기 때문에 훈련 시간을 크게 단축시킬 수 있다는 장점도 가지고 있습니다. 

또한, Transformer는 **구문 분석과 같은 다른 NLP 작업에도 효과적으로 적용**될 수 있음을 보여주었습니다. 

이 논문은 기계 번역 분야에 큰 영향을 미쳤으며, 이후 다양한 NLP 모델에 Attention 메커니즘이 적용되는 계기가 되었습니다. 

Transformer는 **자연어 처리 분야의 새로운 지평을 열었으며, 이후 연구와 개발에 큰 영감을 주었습니다.**





해당 방법은 매우 간단하지만, Context 길이를 넘어서는 경우 에러가 발생합니다.

## Map-Reduce
LangChain의 PyMuPdfLoader를 이용하여 임의의 PDF를 요약해 보겠습니다.

In [6]:
# # Password 있는 PDF 열기
# from langchain.document_loaders import PyPDFLoader
path_material = './교재.pdf'

# pypdf_loader = PyPDFLoader(path_material, password='비밀번호')
# material_pages = pypdf_loader.load()

In [6]:
from langchain_community.document_loaders import PyMuPDFLoader
import urllib.request

paper_URL = "https://arxiv.org/pdf/2410.05983"


# 외부 링크에서 PDF 파일을 다운로드하는 코드
urllib.request.urlretrieve(
    paper_URL,
    filename="paper.pdf"
)
path = './paper.pdf'

In [7]:
from langchain_core.documents import Document


loader = PyMuPDFLoader(path)
# 페이지별로 저장
pages = loader.load()

# 코퍼스에 모두 결합
corpus = Document(page_content='')
for page in pages:
    corpus.page_content += page.page_content
len(corpus.page_content)

97331

긴 Context를 처리하기 위해, 전체 코퍼스를 작은 단위로 쪼개 보겠습니다.

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=1000) #chunk_size는 한 chunk에 포함될 최대 글자수입니다.
document_list = text_splitter.split_documents([corpus])
len(document_list)

11

이후 Map-Reduce와 Refine을 사용할 수 있습니다.

## Map-Reduce

In [9]:
from tqdm import tqdm

# Map 과정 : 각 문서에 대해 요약을 생성합니다. -> Long Context 인 경우, 균일하게 분할하여 요약하고 다시 합치는 방법으로 성능 개선

map_prompt = ChatPromptTemplate.from_messages([
    ('system', '''주어진 논문의 내용을 읽고 한국어로 요약하세요.
요약은 1개의 문단과 문단별 6개의 문장으로 작성하세요.
답변은 한국어로 작성하세요.
같은 내용을 반복하지 마세요.
'''),
    ('user', '''{text}''')])

map_chain  = map_prompt | gemma2 | StrOutputParser()

raw_summaries = []
for i in tqdm(range(len(document_list))):
    response = map_chain.invoke(document_list[i].page_content)
    raw_summaries.append(response)
    print('')
    print('#',i)
    print(response)
    print('===========================')


  9%|▉         | 1/11 [00:00<00:08,  1.14it/s]


# 0
본 논문은 장문맥 LLM(Long-Context LLMs)을 사용한 Retrieval-Augmented Generation(RAG) 시스템의 새로운 과제와 해결책을 제시합니다. 장문맥 LLM은 더 많은 정보를 처리할 수 있지만, 과도한 정보량은 LLM의 생성 성능 저하로 이어질 수 있습니다. 특히, 검색된 "hard negatives" (관련성이 낮은 정보)가 LLM의 생성을 혼란스럽게 만들 수 있습니다. 이 논문에서는 장문맥 LLM을 사용한 RAG 시스템에서 hard negatives의 부정적인 영향을 분석하고, 이를 해결하기 위한 세 가지 새로운 방법을 제안합니다. 첫째, 검색된 문서의 순서를 재정렬하여 LLM이 더욱 관련성 있는 정보에 집중하도록 유도하는 방법입니다. 둘째, hard negatives에 대한 내성을 갖도록 LLM을 훈련하는 방법입니다. 셋째, LLM이 검색된 문서를 분석하고 관련 정보를 명시적으로 식별하도록 훈련하는 방법입니다. 이러한 방법들은 데이터 분포, 사용된 검색 알고리즘, 훈련 문맥 길이 등 다양한 요소를 고려하여 장문맥 LLM 기반 RAG 시스템의 성능을 향상시키는 데 기여합니다.





 18%|█▊        | 2/11 [00:02<00:09,  1.04s/it]


# 1
본 논문은 장문맥 LLM(Long-Context LLMs)을 기반으로 한 RAG(Retrieval-Augmented Generation) 시스템에서 발생하는 과제들을 분석하고 해결 방안을 제시합니다. 

첫째, 장문맥 LLM이 처리할 수 있는 답변 텍스트의 길이가 증가함에 따라 RAG 성능이 향상되는지, 또는 오히려 감소하는지에 대한 질문을 던지며, 다양한 LLM과 검색기(retriever)를 사용하여 실험을 진행합니다. 실험 결과, 검색기의 성능이 높을수록 장문맥 LLM의 성능은 초기에는 증가하지만, 특정 범위를 넘어서면 감소하는 "역U자형" 패턴을 보이는 것으로 나타났습니다. 이는 검색기가 너무 많은 정보를 가져오면 LLM이 오히려 혼란스러워지고 성능이 저하될 수 있음을 시사합니다.

둘째, 검색기의 성능과 LLM의 처리 능력 사이의 상호 작용을 분석합니다. 높은 재현율(recall)을 가진 검색기와 낮은 재현율을 가진 검색기 모두 사용하여 실험을 진행하고, LLM의 성능과 검색 결과의 재현율, 정확도(precision) 사이의 관계를 살펴봅니다. 결과적으로, 높은 재현율을 가진 검색기는 더 많은 정보를 가져오지만, 그 정보 중 일부가 오히려 LLM의 성능을 저하시키는 "hard negatives"로 작용하는 것으로 나타났습니다. 즉, 단순히 검색 결과의 양이 많을수록 LLM의 성능이 향상되는 것은 아니며, 검색 결과의 질과 LLM의 처리 능력 사이의 균형이 중요합니다.

이러한 분석을 통해 논문은 장문맥 LLM을 활용한 RAG 시스템에서 발생하는 핵심 과제들을 명확히 제시하고, 이를 해결하기 위한 방향을 제시합니다. 





 27%|██▋       | 3/11 [00:03<00:08,  1.07s/it]

...


# 8
본 논문은 Long-Context LLMs를 활용한 RAG(Retrieval Augmented Generation) 모델의 성능 향상을 다룹니다. 특히, 중간 단계의 추론 능력을 강화하여 질문에 대한 답변을 생성하는 데 초점을 맞춥니다. 

본 논문은 다양한 데이터셋(NQ, Wizard of Wikipedia, FEVER, MMLU)을 사용하여 RAG 모델을 학습하고 평가합니다. 학습 과정에서, 모델은 질문과 관련된 문서를 분석하고, 답변을 생성하기 위한 논리적 근거를 제시하는 능력을 향상시키도록 지도 학습됩니다. 

평가 결과, 중간 단계 추론을 활용한 RAG 모델은 기존의 RAG 모델보다 질문에 대한 답변의 정확도와 완전성을 향상시키는 것을 보여줍니다. 

특히, 긴 텍스트 입력에 대한 처리 능력이 향상되었으며, 복잡한 질문에 대한 답변을 생성하는 데에도 효과적임을 확인했습니다. 

논문은 또한 데이터 증강 기법을 활용하여 RAG 모델의 성능을 더욱 향상시킬 수 있는 가능성을 제시합니다. 

결론적으로, 본 논문은 Long-Context LLMs를 활용한 RAG 모델의 성능 향상을 위한 새로운 접근 방식을 제시하며, 긴 텍스트 입력 처리 및 복잡한 질문 답변에 대한 잠재력을 보여줍니다.





 91%|█████████ | 10/11 [01:08<00:10, 10.38s/it]


# 9
본 논문은 Gemma-2-9B와 같은 대규모 언어 모델(LLM)을 사용하여 질의응답(RAG) 시스템을 개선하는 방법에 대해 다룹니다. 특히, 데이터 증강된 RAG fine-tuning 기법이 LLM의 성능을 향상시키는 데 효과적임을 보여줍니다. 

첫째, 논문은 다양한 데이터셋에서 Gemma-2-9B 모델을 RAG fine-tuning하여 그 성능을 평가합니다. RAG fine-tuning은 일반적인 챗 모델보다 질의응답 작업에서 더 높은 정확도를 보입니다. 둘째, 논문은 RAG fine-tuning에 중간 단계의 추론을 추가하여 모델의 성능을 더욱 향상시키는 방법을 제시합니다. 이 방법은 복잡한 질문에 대한 답변을 생성하는 데 유용합니다. 셋째, 논문은 Mistral-Nemo-12B 모델과 같은 다른 LLM에서도 데이터 증강된 RAG fine-tuning 기법이 효과적임을 보여줍니다. 넷째, 논문은 RAG fine-tuning이 LLM의 장점을 활용하여 질의응답 시스템의 성능을 향상시키는 데 유용한 방법임을 강조합니다. 마지막으로, 논문은 앞으로의 연구 방향으로 데이터 증강 기법의 다양한 적용 및 더욱 복잡한 질문에 대한 답변을 생성하는 방법에 대한 연구를 제안합니다.





100%|██████████| 11/11 [01:21<00:00,  7.39s/it]


# 10
본 논문은 긴 텍스트 입력에 대한 RAG(Retrieval Augmented Generation)의 효율성을 높이기 위해 Long-Context LLM(Large Language Model)을 사용하는 방법을 연구합니다. 

첫째, Mistral-Nemo-12B와 Gemini-1.0-Pro 모델을 사용하여 RAG-specific fine-tuning의 효과를 보여줍니다. 

둘째, RAG-specific fine-tuning 데이터의 크기가 LLM 성능에 미치는 영향을 분석하여 데이터 규모가 증가함에 따라 성능이 향상됨을 확인합니다. 

셋째, 일반적인 SFT(Supervised Fine-Tuning) 데이터와 RAG-specific 데이터를 결합하여 LLM을 학습시키고, 이 방법이 RAG 성능을 향상시키면서도 일반적인 언어 처리 능력을 유지할 수 있음을 보여줍니다. 

결론적으로, 본 논문은 긴 텍스트 입력을 처리하는 RAG 시스템에서 Long-Context LLM을 활용하고, RAG-specific fine-tuning과 데이터 규모 조절을 통해 성능을 향상시킬 수 있는 방법을 제시합니다. 





In [10]:
raw_summaries

['본 논문은 장문맥 LLM(Long-Context LLMs)을 사용한 Retrieval-Augmented Generation(RAG) 시스템의 새로운 과제와 해결책을 제시합니다. 장문맥 LLM은 더 많은 정보를 처리할 수 있지만, 과도한 정보량은 LLM의 생성 성능 저하로 이어질 수 있습니다. 특히, 검색된 "hard negatives" (관련성이 낮은 정보)가 LLM의 생성을 혼란스럽게 만들 수 있습니다. 이 논문에서는 장문맥 LLM을 사용한 RAG 시스템에서 hard negatives의 부정적인 영향을 분석하고, 이를 해결하기 위한 세 가지 새로운 방법을 제안합니다. 첫째, 검색된 문서의 순서를 재정렬하여 LLM이 더욱 관련성 있는 정보에 집중하도록 유도하는 방법입니다. 둘째, hard negatives에 대한 내성을 갖도록 LLM을 훈련하는 방법입니다. 셋째, LLM이 검색된 문서를 분석하고 관련 정보를 명시적으로 식별하도록 훈련하는 방법입니다. 이러한 방법들은 데이터 분포, 사용된 검색 알고리즘, 훈련 문맥 길이 등 다양한 요소를 고려하여 장문맥 LLM 기반 RAG 시스템의 성능을 향상시키는 데 기여합니다.\n\n\n',
 '본 논문은 장문맥 LLM(Long-Context LLMs)을 기반으로 한 RAG(Retrieval-Augmented Generation) 시스템에서 발생하는 과제들을 분석하고 해결 방안을 제시합니다. \n\n첫째, 장문맥 LLM이 처리할 수 있는 답변 텍스트의 길이가 증가함에 따라 RAG 성능이 향상되는지, 또는 오히려 감소하는지에 대한 질문을 던지며, 다양한 LLM과 검색기(retriever)를 사용하여 실험을 진행합니다. 실험 결과, 검색기의 성능이 높을수록 장문맥 LLM의 성능은 초기에는 증가하지만, 특정 범위를 넘어서면 감소하는 "역U자형" 패턴을 보이는 것으로 나타났습니다. 이는 검색기가 너무 많은 정보를 가져오면 LLM이 오히려 혼란스러워지고 성능이 저하될 수 있음을 시사합니다.\n\n둘째, 검색기의 성능과 LLM의 처

In [11]:
# Reduce 과정 : 각 문서의 요약을 하나로 합칩니다.
reduce_prompt = ChatPromptTemplate.from_messages([

    ('system', '''
Generate a summary of the following text that includes the following elements:

* A title that accurately reflects the content of the text.
* An introduction paragraph that provides an overview of the topic.
* Bullet points that list the key points of the text.
* A conclusion paragraph that summarizes the main points of the text.

Answer in Korean.
'''),
    ('user', '''{text}
---
요약(In Korean):
''')])

reduce_chain = reduce_prompt | gemma2 | StrOutputParser()

summary = reduce_chain.invoke('\n---\n'.join(raw_summaries))

print(summary)


## 장문 맥락 LLM을 활용한 RAG 시스템 성능 향상 방안 연구

본 논문은 장문 맥락을 가진 대규모 언어 모델(LLM)을 Retrieval-Augmented Generation(RAG) 시스템에 적용하여 성능을 향상시키는 방법을 연구합니다. 

**핵심 내용:**

* **장문 맥락 LLM의 RAG 적용 문제점:** 장문 맥락 LLM은 많은 정보를 처리해야 하기 때문에, 검색된 텍스트의 양과 질이 RAG 성능에 큰 영향을 미칩니다. 특히, 강력한 검색 알고리즘으로 검색된 "hard negatives" (정답과 관련 없는 텍스트)는 LLM의 성능을 저하시킬 수 있습니다.
* **해결 방안:** 
    * 검색된 문서의 순서를 재정렬하여 LLM이 관련성 높은 정보에 집중하도록 유도
    * hard negatives에 대한 내성을 갖도록 LLM을 훈련
    * LLM이 검색된 문서를 분석하고 관련 정보를 명시적으로 식별하도록 훈련
* **실험 결과:** 제안된 방법들은 다양한 데이터셋과 검색 알고리즘에서 성능 향상을 보여주며, 특히 긴 텍스트 입력에 대한 처리 능력을 향상시키는 데 효과적입니다.

**결론:**

장문 맥락 LLM을 RAG 시스템에 적용하면 긴 텍스트 입력 처리 능력을 향상시킬 수 있으며, 데이터 분포, 사용된 검색 알고리즘, 훈련 텍스트 길이 등 다양한 요소를 고려하여 성능을 극대화할 수 있습니다.







## Refine

Refine은 청크를 순서대로 참고하며, 매 시점 요약문을 작성합니다.   
요약문과 새로운 청크를 비교하여, 요약문을 업데이트합니다.

In [12]:
# 청크 순서대로 각 청크들을 요약하고, 앞부분 청크 요약과 현재 청크를 연결하여 요약을 업데이트하며 진행한다.
# 시간은 오래 걸릴 수 있으나, 청크 업데이트가 잦은 경우에 사용
first_summarize_prompt = ChatPromptTemplate.from_messages([
    ('system', '''당신은 인공지능 전문가입니다.
주어진 논문의 내용을 읽고 한국어로 요약하세요.
요약은 1개의 문단과 문단별 5개의 문장으로 작성하세요.
답변은 한국어로 작성하세요.'''),
    ('user', '''{text}''')])



X = first_summarize_prompt.format_messages(text=document_list[0])

intermediate_summary = gemma2.invoke(X).content
print(intermediate_summary)

본 논문은 장문 맥락을 가진 대규모 언어 모델(LLM)을 사용하는 Retrieval-Augmented Generation(RAG) 시스템의 문제점과 해결책을 제시합니다. 

장문 맥락 LLM은 더 많은 정보를 활용할 수 있어 성능 향상을 기대할 수 있지만, 실험 결과는 그렇지 않은 경우가 많았습니다. 

연구 결과, 많은 장문 맥락 LLM에서 검색된 정보의 양이 증가할수록 생성된 출력의 품질은 처음에는 향상되지만, 이후 감소하는 경향을 보입니다. 이는 검색된 "hard negatives" (관련성이 낮은 정보)가 LLM 생성에 부정적인 영향을 미치기 때문으로 분석되었습니다. 

이러한 문제를 해결하기 위해 논문에서는 학습 없이 검색된 문서 순서를 재정렬하는 방법과, LLM을 학습시켜 hard negatives에 대한 내성을 높이는 방법, 그리고 LLM이 검색된 정보 중 관련성 있는 정보를 명시적으로 식별하도록 학습시키는 방법을 제안합니다. 

결론적으로, 본 논문은 장문 맥락 LLM을 사용하는 RAG 시스템에서 발생하는 새로운 문제점을 분석하고, 이를 해결하기 위한 효과적인 방법을 제시합니다.





In [13]:
# Refine Prompt

refine_prompt = ChatPromptTemplate.from_messages([
    ('system', '''당신은 인공지능 전문가입니다.
논문의 현재 시점까지의 한국어 요약이 주어집니다.
이를 읽고, 새롭게 주어지는 내용과 비교하여 한국어 요약을 보완하거나 수정하세요.
전체 요약은 10문장 이내로 작성하세요.
'''),
    ('user', '''현재 시점까지의 요약: {previous_summary}
---
새로운 내용: {new_text}''')])


refine_chain = refine_prompt | gemma2 | StrOutputParser()

for i in tqdm(range(1, len(document_list))):
    intermediate_summary = refine_chain.invoke(
        {'previous_summary':intermediate_summary,
         'new_text':document_list[i].page_content})
    print('')
    print(intermediate_summary)
    print('=======================')
# 길이를 지정하지 않으면 오래 걸릴 수 있습니다.



 10%|█         | 1/10 [00:00<00:08,  1.03it/s]


본 논문은 장문 맥락을 가진 대규모 언어 모델(LLM)을 사용하는 Retrieval-Augmented Generation(RAG) 시스템의 문제점과 해결책을 제시합니다. 장문 맥락 LLM은 더 많은 정보를 활용할 수 있어 성능 향상을 기대할 수 있지만, 실험 결과는 그렇지 않은 경우가 많았습니다. 

연구 결과, 많은 장문 맥락 LLM에서 검색된 정보의 양이 증가할수록 생성된 출력의 품질은 처음에는 향상되지만, 이후 감소하는 경향을 보입니다. 이는 검색된 "hard negatives" (관련성이 낮은 정보)가 LLM 생성에 부정적인 영향을 미치기 때문으로 분석되었습니다. 

특히, 강력한 검색 알고리즘이 더 많은 관련 정보를 찾아내더라도, 그 안에 포함된 "hard negatives"가 LLM의 성능을 저해하는 문제점이 발견되었습니다. 이는 단순히 검색된 정보의 양이 많을수록 성능이 향상되는 것은 아니며, 정보의 질과 관련성이 중요함을 시사합니다.

이러한 문제를 해결하기 위해 논문에서는 학습 없이 검색된 문서 순서를 재정렬하는 방법과, LLM을 학습시켜 hard negatives에 대한 내성을 높이는 방법, 그리고 LLM이 검색된 정보 중 관련성 있는 정보를 명시적으로 식별하도록 학습시키는 방법을 제안합니다. 






 20%|██        | 2/10 [00:09<00:42,  5.28s/it]


본 논문은 장문 맥락을 가진 대규모 언어 모델(LLM)을 사용하는 Retrieval-Augmented Generation(RAG) 시스템의 문제점과 해결책을 제시합니다. 장문 맥락 LLM은 더 많은 정보를 활용할 수 있어 성능 향상을 기대할 수 있지만, 실험 결과 그렇지 않은 경우가 많았습니다. 

연구 결과, 많은 장문 맥락 LLM에서 검색된 정보의 양이 증가할수록 생성된 출력의 품질은 처음에는 향상되지만, 이후 감소하는 경향을 보입니다. 이는 검색된 "hard negatives" (관련성이 낮은 정보)가 LLM 생성에 부정적인 영향을 미치기 때문으로 분석되었습니다. 특히, 강력한 검색 알고리즘이 더 많은 관련 정보를 찾아내더라도, 그 안에 포함된 "hard negatives"가 LLM의 성능을 저해하는 문제점이 발견되었습니다. 

단순히 검색된 정보의 양이 많을수록 성능이 향상되는 것은 아니며, 정보의 질과 관련성이 중요함을 시사합니다. 이러한 문제를 해결하기 위해 논문에서는 학습 없이 검색된 문서 순서를 재정렬하는 방법과, LLM을 학습시켜 hard negatives에 대한 내성을 높이는 방법, 그리고 LLM이 검색된 정보 중 관련성 있는 정보를 명시적으로 식별하도록 학습시키는 방법을 제안합니다. 

새로운 내용에 따르면, "hard negatives"는 LLM의 성능에 큰 영향을 미치며, 특히 강력한 검색 알고리즘이 사용될수록 더욱 심각해집니다. 기존의 평가 방법은 주로 무작위 음성을 사용하기 때문에, 실제 RAG 시스템에서 발생하는 "hard negatives" 문제를 충분히 반영하지 못합니다. 따라서, "hard negatives"를 고려한 새로운 평가 방법이 필요합니다. 





 30%|███       | 3/10 [00:24<01:09,  9.99s/it]

...


## 장문 맥락 LLM을 활용한 RAG 시스템의 문제점과 해결 방안: 새로운 내용 반영

본 논문은 장문 맥락을 가진 대규모 언어 모델(LLM)을 사용하는 Retrieval-Augmented Generation(RAG) 시스템의 문제점과 해결책을 제시합니다. 장문 맥락 LLM은 더 많은 정보를 활용하여 성능 향상을 기대할 수 있지만, 실험 결과 그렇지 않은 경우가 많았습니다. 

연구 결과, 많은 장문 맥락 LLM에서 검색된 정보의 양이 증가할수록 생성된 출력의 품질은 처음에는 향상되지만, 이후 감소하는 경향을 보입니다. 이는 검색된 "hard negatives" (관련성이 낮은 정보)가 LLM 생성에 부정적인 영향을 미치기 때문으로 분석되었습니다. 특히, 강력한 검색 알고리즘이 더 많은 관련 정보를 찾아내더라도 그 안에 포함된 "hard negatives"가 LLM의 성능을 저해하는 문제점이 발견되었습니다. 

새로운 연구 결과에 따르면, "hard negatives"는 LLM의 성능에 큰 영향을 미치며, 특히 강력한 검색 알고리즘이 사용될수록 더욱 심각해집니다. 기존의 평가 방법은 주로 무작위 음성을 사용하기 때문에, 실제 RAG 시스템에서 발생하는 "hard negatives" 문제를 충분히 반영하지 못합니다. 따라서, "hard negatives"를 고려한 새로운 평가 방법이 필요합니다. 

단순히 검색된 정보의 양이 많을수록 성능이 향상되는 것은 아니며, 정보의 질과 관련성이 중요함을 시사합니다. 이러한 문제를 해결하기 위해 논문에서는 학습 없이 검색된 문서 순서를 재정렬하는 방법, LLM을 학습시켜 hard negatives에 대한 내성을 높이는 방법, 그리고 LLM이 검색된 정보 중 관련성 있는 정보를 명시적으로 식별하도록 학습시키는 방법을 제안합니다. 

**추가적으로, 논문에서는 다양한 데이터셋과 평가 방법을 사용하여 실험을 진행했습니다.** 

* **훈련 데이터셋:** Natural Question, Wizard of Wikipedia, 

 90%|█████████ | 9/10 [02:08<00:17, 17.20s/it]


## 장문 맥락 LLM을 활용한 RAG 시스템의 문제점과 해결 방안: 새로운 내용 반영

본 논문은 장문 맥락을 가진 대규모 언어 모델(LLM)을 사용하는 Retrieval-Augmented Generation(RAG) 시스템의 문제점과 해결책을 제시합니다. 장문 맥락 LLM은 더 많은 정보를 활용하여 성능 향상을 기대할 수 있지만, 실험 결과 그렇지 않은 경우가 많았습니다. 

연구 결과, 많은 장문 맥락 LLM에서 검색된 정보의 양이 증가할수록 생성된 출력의 품질은 처음에는 향상되지만, 이후 감소하는 경향을 보입니다. 이는 검색된 "hard negatives" (관련성이 낮은 정보)가 LLM 생성에 부정적인 영향을 미치기 때문으로 분석되었습니다. 특히, 강력한 검색 알고리즘이 더 많은 관련 정보를 찾아내더라도 그 안에 포함된 "hard negatives"가 LLM의 성능을 저해하는 문제점이 발견되었습니다. 

새로운 연구 결과에 따르면, "hard negatives"는 LLM의 성능에 큰 영향을 미치며, 특히 강력한 검색 알고리즘이 사용될수록 더욱 심각해집니다. 기존의 평가 방법은 주로 무작위 음성을 사용하기 때문에, 실제 RAG 시스템에서 발생하는 "hard negatives" 문제를 충분히 반영하지 못합니다. 따라서, "hard negatives"를 고려한 새로운 평가 방법이 필요합니다. 

단순히 검색된 정보의 양이 많을수록 성능이 향상되는 것은 아닙니다. 정보의 질과 관련성이 중요함을 시사합니다. 이러한 문제를 해결하기 위해 논문에서는 학습 없이 검색된 문서 순서를 재정렬하는 방법, LLM을 학습시켜 hard negatives에 대한 내성을 높이는 방법, 그리고 LLM이 검색된 정보 중 관련성 있는 정보를 명시적으로 식별하도록 학습시키는 방법을 제안합니다. 

**추가적으로, 논문에서는 다양한 데이터셋과 평가 방법을 사용하여 실험을 진행했습니다.** 

* **훈련 데이터셋:** Natural Question, Wizard of Wikipedia,

100%|██████████| 10/10 [02:35<00:00, 15.59s/it]


## 장문 맥락 LLM을 활용한 RAG 시스템의 문제점과 해결 방안: 새로운 내용 반영

본 논문은 장문 맥락을 가진 대규모 언어 모델(LLM)을 사용하는 Retrieval-Augmented Generation(RAG) 시스템의 문제점과 해결책을 제시합니다. 

장문 맥락 LLM은 더 많은 정보를 활용하여 성능 향상을 기대할 수 있지만, 실험 결과 그렇지 않은 경우가 많았습니다. 특히, 검색된 정보의 양이 증가할수록 생성된 출력의 품질이 처음에는 향상되지만, 이후 감소하는 경향을 보이는 문제점이 발견되었습니다. 이는 검색된 "hard negatives" (관련성이 낮은 정보)가 LLM 생성에 부정적인 영향을 미치기 때문으로 분석되었습니다. 

새로운 연구 결과에 따르면, "hard negatives"는 LLM의 성능에 큰 영향을 미치며, 특히 강력한 검색 알고리즘이 사용될수록 더욱 심각해집니다. 기존의 평가 방법은 주로 무작위 음성을 사용하기 때문에, 실제 RAG 시스템에서 발생하는 "hard negatives" 문제를 충분히 반영하지 못합니다. 따라서, "hard negatives"를 고려한 새로운 평가 방법이 필요합니다. 

단순히 검색된 정보의 양이 많을수록 성능이 향상되는 것은 아닙니다. 정보의 질과 관련성이 중요함을 시사합니다. 이러한 문제를 해결하기 위해 논문에서는 학습 없이 검색된 문서 순서를 재정렬하는 방법, LLM을 학습시켜 hard negatives에 대한 내성을 높이는 방법, 그리고 LLM이 검색된 정보 중 관련성 있는 정보를 명시적으로 식별하도록 학습시키는 방법을 제안합니다. 

논문에서는 다양한 데이터셋과 평가 방법을 사용하여 실험을 진행했습니다. 

* **훈련 데이터셋:** Natural Question, Wizard of Wikipedia, FEVER, MMLU 등 다양한 유형의 데이터셋을 사용하여 LLM의 성능을 향상시키는 방법을 연구했습니다.
* **테스트 데이터셋:** TriviaQA, PopQA, WebQuestions, 